In [ ]:
#!pip3 install langchain_community
#!pip3 install langchain_openai
#pip3 install bert_score

In [1]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os
import ast
from openai import AzureOpenAI
import faiss
import numpy as np
import json
from bert_score import score




### Load Enviroment files

In [2]:
# Load environment variables
load_dotenv()

Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 12
Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 15
Python-dotenv could not parse statement starting at line 16
Python-dotenv could not parse statement starting at line 17
Python-dotenv could not parse statement starting at line 18
Python-dotenv could not parse statement starting at line 19
Python-dotenv could not parse statement starting at line 20


True

In [6]:
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai_deployment = "sdgi-gpt-35-turbo-16k"
neo4j_pass = os.getenv("NEO4JPASS")


In [7]:
# OpenAI API configuration
#openai.api_type = "azure"
#openai.api_key = os.getenv("api_key_azure")
#openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
#openai.api_version = os.getenv("api_version")
#openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("OPENAI_API_KEY"),  
  api_version = os.getenv("OPENAI_API_VERSION"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

embedding_model = os.getenv("USER_QUERY_EMBEDDING_ENGINE") 

# print(openai.api_key)
# print(openai.api_base)
# print(openai.api_version)


In [46]:
with open("queries.json") as f:
    data = f.read()
    query_list = json.loads(data)
    

<h3>globals</h3>

In [8]:
test_query = query_list['query1']


<h3> helper functions </h3>

In [9]:
# use this function to make simple openAI Calls
def callOpenAI(prompt):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    response = response_entities.choices[0].message.content
    return response


<h3> processing modules </h3>

In [10]:
def extractEntitiesFromQuery(user_query):
    prompt = f"""
    Extract entities from the following user query: \"{user_query}\" and return output in array format.
    
    -Entities should be directly related to the domain or topic of interest. They should represent important concepts that contribute to the understanding of the subject matter.
    -Each entity in the knowledge graph should be distinct and have a unique identifier. This ensures clarity and avoids ambiguity when establishing relationships between entities.
    -You Must return output in array format e.g  ['entity1','entity2'] !!!
    -Avoid adding new lines or breaking spaces to your output. Array should be single dimension and single line !!!
    """
    entity_str = callOpenAI(prompt)   
    return entity_str



In [11]:
entity_str = extractEntitiesFromQuery(test_query)
# Convert the string representation of the list to an actual list
entity_list = ast.literal_eval(entity_str)

type(entity_list)

list

# KG module

In [12]:
from py2neo import Graph, Node, Relationship
graph = Graph(uri = 'bolt://localhost:7687',user='neo4j',password=neo4j_pass)

In [13]:
def search_entities_with_relations(label, name):
    """
    Search for nodes by label and name, including all their relationships and connected nodes.

    :param label: The label of the nodes to search.
    :param name: The name property value of the nodes to search.
    :return: A list of dictionaries, each containing a node and its relationships and connected nodes.
    """
    # Match nodes and their relationships, case-insensitive search
    query = f"""
    MATCH (n:`{label}`)-[r]-(m)
    WHERE toLower(n.name) = toLower($name) OR toLower(n.acronym) = toLower($name)
    RETURN n, collect(r) as relations, collect(m) as connectedNodes
    """
    results = graph.run(query, name=name).data()
    
    # Construct a comprehensive view for each node with its relationships and connected nodes
    entities_with_relations = []
    for record in results:
        entity_info = {
            'node': record['n'],
            'relationships': record['relations'],
            'connected_nodes': record['connectedNodes']
        }
        entities_with_relations.append(entity_info)

    return entities_with_relations


In [36]:
def separate_entities_and_relationships(relationships):
    """
    Separates entities and relationships from Neo4j query output into distinct structures,
    ensuring no duplicate entities are included.

    :param relationships: A list of Relationship objects from a Neo4j query.
    :return: A tuple containing two lists: one for unique entities and another for relationships.
    """
    # Using a dictionary to ensure unique entities based on a combination of name and category
    entities_dict = {}
    rels = []

    for rel in relationships:
        # Process both the source and target nodes for each relationship
        for node in [rel.start_node, rel.end_node]:
            # Define a unique identifier for each entity
            entity_id = f"{node['name']}_{node['category']}"
            
            # If the entity is not already in the dictionary, add it
            if entity_id not in entities_dict:
                entities_dict[entity_id] = {
                    'name': node['name'],
                    'category': node.get('category', 'N/A'),
                    #'summary': node.get('summary', 'N/A'),
                    'acronym': node.get('acronym', 'N/A'),
                }

        # Add relationship information
        rels.append({
            'subject': rel.start_node['name'],
            'object': rel.end_node['name'],
            'relationship_type': rel.__class__.__name__,
            'description': rel.get('description', 'N/A'),
        })

    # Convert the entities dictionary to a list to remove the unique identifier layer
    entities_list = list(entities_dict.values())

    return entities_list, rels




In [17]:
def search_embeddings(user_query):
    df = pd.read_pickle('../models/df_embed_EN.pkl')
    df_filtered = df
    length = len(df_filtered.head())
    filtered_embeddings_arrays = np.array(list(df_filtered['Embedding']))
    index = faiss.IndexFlatIP(filtered_embeddings_arrays.shape[1]) 
    index.add(filtered_embeddings_arrays)
    
    user_query_embedding = client.embeddings.create( 
        input=user_query ,model= embedding_model
    ).data[0].embedding
    
    k = min(5, length)
    distances, indices = index.search(np.array([user_query_embedding]), k)
    return df_filtered, distances, indices

In [18]:
def get_answer(user_question, content):
    system_prompt = "You are a system that answers user questions based on excerpts from PDF documents provided for context. Only answer if the answer can be found in the provided context. Do not make up the answer; if you cannot find the answer, say so."
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_question},
        {'role': 'user', 'content': content},
    ]
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0.2,
                    messages=messages
                )
    response = response_entities.choices[0].message.content
    return response
  

In [20]:
def map_to_structure(qs):
    result_dict = {}

    # Extract the DataFrame from the tuple
    dataframe = qs[0]

    # Counter to limit the loop to 10 iterations
    count = 0

    for index, row in dataframe.iterrows():
        # Define a unique identifier for each document, you can customize this based on your data
        document_id = f"doc-{index + 1}"
        # Handle NaN in content by using fillna
        content = row["Content"]
        content = ' '.join(row["Content"].split()[:160])
        # Create a dictionary for each document
        document_info = {
            "title": row["Document Title"],
            "extract": content or "",  # You may need to adjust this based on your column names
            "category": row["Category"],
            "link": row["Link"],
            "thumbnail": ''
        }
        # print(document_info)
        # Add the document to the result dictionary
        result_dict[document_id] = document_info

        # Increment the counter
        count += 1

        # # Break out of the loop if the counter reaches top 10
        if count == 10:
            break

    return result_dict


In [29]:
## module to get data for specific indicators which are identified is relevant to the user query

def indicatorsModule(user_query): #lower priority
    
    # find relevant indicators based on uesr query and extract values
    indicators_dict={
        "indicator-id-1":"value from indicator-id-1",
        "indicator-id-2":"value from indicator-id-2"
    }#temp
    
    return indicators_dict

## module to extract text from documents and return the text and document codes

def semanticSearchModule(user_query):
    qs = search_embeddings(user_query) #df, distances, indices
    result_structure = map_to_structure(qs)
    return result_structure


In [21]:
relations_list = []

In [37]:
for item in entity_list:
    entities_with_relations = search_entities_with_relations('Entity', item)
    
    for entity_info in entities_with_relations:
        relations_list.extend(entity_info['relationships'])
        #print("Connected Nodes:", entity_info['connected_nodes'])

# Assuming 'relationships' is your list of Relationship objects from the query
entities, relationships = separate_entities_and_relationships(relations_list)


In [38]:
relationships

[{'subject': 'Gender Development Index',
  'object': 'Albania',
  'relationship_type': 'focuses_on',
  'description': 'Albania has a Gender Development Index of 0.967.'},
 {'subject': 'United Nations Development Programme',
  'object': 'United Nations agencies',
  'relationship_type': 'partners_with',
  'description': 'UNDP partners with United Nations agencies to achieve its goals.'},
 {'subject': 'United Nations',
  'object': 'United Nations Development Programme',
  'relationship_type': 'partners_with',
  'description': 'Building on previous successes, UNDP will work with civil society and the government to improve human capacities and data systems for performance, transparency and planning. UNDP will work on the quality of services and rule of law, in close collaboration with United Nations partners.'},
 {'subject': 'United Nations Population Fund',
  'object': 'United Nations Development Programme',
  'relationship_type': 'partners_with',
  'description': 'The United Nations Popul

In [39]:
entities

[{'name': 'Gender Development Index',
  'category': 'Strategy',
  'acronym': 'GDI'},
 {'name': 'Albania', 'category': 'Location', 'acronym': 'N/A'},
 {'name': 'United Nations Development Programme',
  'category': 'Organization',
  'acronym': 'UNDP'},
 {'name': 'United Nations agencies',
  'category': 'Organization',
  'acronym': 'N/A'},
 {'name': 'United Nations', 'category': 'Organization', 'acronym': 'UN'},
 {'name': 'United Nations Population Fund',
  'category': 'Organization',
  'acronym': 'UNFPA'},
 {'name': 'Government', 'category': 'Organization', 'acronym': 'Government'},
 {'name': 'Sustainable Development Goals',
  'category': 'Strategy',
  'acronym': 'SDGs'},
 {'name': 'United Nations Office for Project Services',
  'category': 'Organization',
  'acronym': 'UNOPS'},
 {'name': 'United Nations Sustainable Development Cooperation Framework',
  'category': 'Organization',
  'acronym': 'UNSDCF'},
 {'name': 'National Statistical Office',
  'category': 'Organization',
  'acronym': 

In [30]:

indicators_dict=indicatorsModule(test_query)
print(indicators_dict)

#test usage
excerpts_dict=semanticSearchModule(test_query)
# print(excerpts_dict)


#Return top 10-20 most related 
# Define the filename to save the JSON data -  can remove later
json_filename = "outputs/excerpts.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(excerpts_dict, json_file, ensure_ascii=False, indent=4)

print(f"Excerpts saved to {json_filename}")

{'indicator-id-1': 'value from indicator-id-1', 'indicator-id-2': 'value from indicator-id-2'}
Excerpts saved to outputs/excerpts.json


<h3> synthesis module </h3>

    llm_instructions="llm instruction template here, with placeholders for insertion of user query, excerpts, indicator data, and entity and relation info" 


In [26]:
# module to synthesize answer using retreival augmented generation approach

def synthesisModule(user_query, excerpts_dict, indicators_dict):
    
    # Generate prompt engineering text and template
    llm_instructions = f"""
    Ignore previous commands!!!
    Given a user query, use the provided excerpts, and Sources to
    provide the correct answer to the user's query
    
    User Query: {user_query}
    
    Sources: {excerpts_dict}
    

    - Answer output must be properly formatted using HTML. 
    - Don't include <html>, <script>, <link> or <body> tags. Only text formatting tags should be allowed. e.g h1..h3, p, anchor, etc.
    - Make sure to Include citations based on the Sources. e.g Text excerpt here<a data-id='test-doc-1'>[1]</a> when referencing a document in the sources. using 1 ...nth
    - The citations anchor should be near the excerpt not following each other.
    - Use the anchor tag for the citation links and should link to the document link. for example UNDP operates in Afghanistan <a data-id='test-doc-1'>[1]</a>. UNDP offers health relationships <a data-id='test-doc-2'>[2]</a>.
    - The text in the anchor tag should be citation number not document title.
    - You can reference multiple citations based sources
    """
    ###synthesize data into structure within llm prompt engineering instructions
    answer=callOpenAI(llm_instructions)
    
    return answer

## to test this, run the full pipeline with the handleApiCall function

In [27]:
# module to synthesize answer using retreival augmented generation approach

def synthesisModule_with_KG(user_query, entities, relationships, excerpts_dict, indicators_dict):
    
    # Generate prompt engineering text and template
    llm_instructions = f"""
    Ignore previous commands!!!
    Given a user query, use the provided excerpts, Sources, and relevant entities and relations to
    provide the correct answer to the user's query
    
    User Query: {user_query}
    
    Sources: {excerpts_dict}
    
    Entity and Relation info: {entities} {relationships}

    - Answer output must be properly formatted using HTML. 
    - Don't include <html>, <script>, <link> or <body> tags. Only text formating tags should be allowed. e.g h1..h3, p, anchor, etc.
    - Make sure to Include citations based on the Sources. e.g Text excerpt here<a data-id='test-doc-1'>[1]</a> when referencing a document in the sources. using 1 ...nth
    - The citations anchor should be near the excerpt not following each other.
    - Use the anchor tag for the citation links and should link to the document link. for example Undp operates in Afghanistan <a data-id='test-doc-1'>[1]</a>. UNDP offers health relationships <a data-id='test-doc-2'>[2]</a>.
    - The text in the anchor tag should be citation number not document title.
    - You can reference multiple citations based sources
    """
    ###synthesize data into structure within llm prompt engineering instructions
    answer=callOpenAI(llm_instructions)
    
    return answer

## to test this, run the full pipeline with the handleApiCall function

<h3> run pipeline </h3>

In [31]:
# full pipeline with retreival, synthesis of answer to user query, and structure results into api response

def handleApiCall(user_query):
        
    ##run processing modules (in parallel)
    #entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    #query_idea_list=queryIdeationModule(user_query) ##lower priority
    
    ##synthesis module
    answer=synthesisModule(user_query, excerpts_dict, indicators_dict)
    
    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict
        
    }
    
    return response

# test usage
response=handleApiCall(test_query) 
# Define the filename to save the JSON data -  can remove later
json_filename = "outputs/query2_answer.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4)

print(f"Synthesis saved to {json_filename}")

Synthesis saved to query2_output.json


In [42]:
# full pipeline with retreival, synthesis of answer to user query, and structure results into api response

def handle_api_call_with_kg(user_query):
        
    ##run processing modules (in parallel)
    #entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    #query_idea_list=queryIdeationModule(user_query) ##lower priority
    
    ##synthesis module
    answer=synthesisModule_with_KG(user_query, entities, relationships[:10], excerpts_dict, indicators_dict)
    
    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict,
        "entities": entities,
        "relations":relationships
        
    }
    
    return response

# test usage
response=handle_api_call_with_kg(test_query) 
# Define the filename to save the JSON data -  can remove later
json_filename = "outputs_KG/Query2_output.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4)

print(f"Synthesis saved to {json_filename}")

Synthesis saved to outputs_KG/Query2_output.json


<h3>testing</h3>

In [16]:
## next step, develop automated testing for all modules
## iterate through test_queries and build automated tests to score results

# open testing dataset with queries and expected results
test_queries_df=pd.read_csv("../testing/energy_ai_test_dataset_v0.csv")



In [ ]:
# Candidate and reference texts
cands = ["The quick brown fox jumps over the lazy dog"]
refs = ["A fast brown fox leaps over a lazy dog"]

# Calculate BERTScore
P, R, F1 = score(cands, refs, lang="en")

# Output the results
print(f"Precisions: {P.tolist()}")
print(f"Recalls: {R.tolist()}")
print(f"F1 scores: {F1.tolist()}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  # TODO::: 
  ##### Add citation prompt to the synthesis module. -done 
  ##### Convert notebook to flask API script. main.py - done
  ##### Refactor PDF -> txt pipeline 

  

In [83]:
## module to get information on the entities from user query using the KG
def knowledgeGraphModule(user_query):
    
    # generate list of entities based on user query
    entity_list = extractEntitiesFromQuery(user_query)
    my_list = ast.literal_eval(entity_list)
    prompt_summarise_entites = f"""
    Summarize all relations between all the entities : {my_list}
    """
    summarise_entities = callOpenAI(prompt_summarise_entites)
    # Initialize an empty dictionary to store information
    entities_dict = {
        "relations": summarise_entities,
        "entities": {}
    }
    # Loop through each entity in the list
    for entity in my_list:
        # Fetch information about the entity from your knowledge graph
        prompt = f"Give me a short description 50 words of {entity}"
        entity_info = callOpenAI(prompt)
        # Add the entity information to the dictionary
        entities_dict["entities"][entity] = entity_info
    
    return entities_dict


# Test usage
test_query = "what is the major work on UNDP in Afganistan?"
entities_dict = knowledgeGraphModule(test_query)
print(entities_dict)

{'relations': 'The relation between UNDP and Afghanistan is that UNDP works in Afghanistan to support development projects and initiatives.', 'entities': {'UNDP': 'The United Nations Development Programme (UNDP) is a global organization that works to eradicate poverty, reduce inequalities, and promote sustainable development. It provides support to countries in areas such as governance, climate change, and crisis response, aiming to improve the lives of people and protect the planet.', 'Afghanistan': 'Afghanistan is a landlocked country located in Central Asia, bordered by Iran, Pakistan, Turkmenistan, Uzbekistan, Tajikistan, and China. It is known for its rugged mountainous terrain, rich cultural heritage, and complex history. Despite facing numerous challenges, Afghanistan is home to resilient people, diverse ethnic groups, and a vibrant blend of traditions.'}}


In [14]:
def relationships_to_data_structure(relationships):
    """
    Converts relationships from Neo4j query output into a structured Python data structure.

    :param relationships: A list of Relationship objects from a Neo4j query.
    :return: A list of dictionaries, each representing a relationship and its node details.
    """
    formatted_relationships = []

    # Sort the relationships by the name of the source node for consistent ordering
    sorted_relationships = sorted(relationships, key=lambda rel: rel.start_node['name'])

    for rel in sorted_relationships:
        relationship_data = {
            'relationship_type': rel.__class__.__name__,
            'source_node': {
                'name': rel.start_node['name'],
                'category': rel.start_node.get('category', 'N/A'),
                'summary': rel.start_node.get('summary', ''),
                'acronym': rel.start_node.get('acronym', ''),
                # Include additional properties as needed
            },
            'target_node': {
                'name': rel.end_node['name'],
                'category': rel.end_node.get('category', ''),
                'summary': rel.end_node.get('summary', ''),
                'acronym': rel.end_node.get('acronym', ''),

                # Include additional properties as needed
            },
            'properties': {
                # Include relationship properties
                'description': rel.get('description', 'N/A'),
            
            }
        }

        formatted_relationships.append(relationship_data)

    return formatted_relationships

# Example usage, assuming 'relationships' is your list of Relationship objects from the query
formatted_data = relationships_to_data_structure(entity_info['relationships'])


In [39]:
def format_and_sort_relationships(relationships):
    """
    Formats and sorts the relationships from Neo4j query output.

    :param relationships: A list of Relationship objects from a Neo4j query.
    """
    # Sort the relationships by the name of the source node for consistent ordering
    sorted_relationships = sorted(relationships, key=lambda rel: rel.start_node['name'])

    for rel in sorted_relationships:
        source_node = rel.start_node
        target_node = rel.end_node
        print(f"Relationship: {rel.__class__.__name__}")
        print(f"  Source Node: {source_node['name']} (Category: {source_node.get('category', 'N/A')})")
        print(f"  Target Node: {target_node['name']} (Category: {target_node.get('category', 'N/A')})")
        
        # Print relationship properties with indentation
        if rel.get('description'):
            print(f"  Description: {rel['description']}")
        if rel.get('summary'):
            print(f"  Summary: {rel['summary']}")

        print()  # Add a newline for spacing between relationships

# Example usage, assuming 'relationships' is your list of Relationship objects from the query
format_and_sort_relationships(entity_info['relationships'])


Relationship: contributes_to
  Source Node: Afghanistan (Category: Location)
  Target Node: Renewable Energy Coordination committee (Category: Organization)
  Description: Given the importance of REN and the role it will play in the socio-economic development of Afghanistan, its development needs to be put on a “fast-track” Renewable Energy Coordination committee which will be established under MEW and work under the Deputy Minister for Energy the mandate of the RECC is to coordinate with all relevant gov/non gov entities for the implementation of this policy.

